# Solar Envelope

In this workshop we will learn how to compute the solar envelop of the building based on a voxelized building envelope.

## README
this notebook contains comments starting with "NOTE:" please read carefully as it contains necessary practical information.

## 0. Initialization
Importing all necessary libraries and specifying the inputs

In [49]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import csv
import pickle
from ladybug.sunpath import Sunpath

## 1. Import Meshes (context + envelope)

### 1.1. Load Meshes

In [2]:
#Loading the envelope and reduced extended context
envelope_path = os.path.relpath("data\envelope.stl")
context_path = os.path.relpath("data\extended_context_reduced.stl")

# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)

True


### 1.2. Visualize Meshes (with pyvista)

In [3]:
# convert trimesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(2883.509098305069, 3262.3801157611238, 2924.4982645648347),
 (-13.99542236328125, 364.87559509277344, 26.993743896484375),
 (0.0, 0.0, 1.0)]

## 2. Import Lattice (envelope)

### 2.1. Load the Envelope Lattice

In [149]:
# loading the current lattice
# NOTE: on first pass load "lattice_14_4" on subsequent passes load "lattice_continue"
lattice_path = os.path.relpath("data\lattice_14_4.csv")
envelope_lattice = tg.lattice_from_csv(lattice_path)
current_res_lattice = envelope_lattice[:]


### 2.2. Visualize the Context Mesh + Envelope Lattice

In [150]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(2883.595633029055, 3262.4666504851098, 2924.5910550347085),
 (-13.99542236328125, 364.87559509277344, 26.99999964237213),
 (0.0, 0.0, 1.0)]

## 3. Sun Vectors

### 3.1. Compute Sun Vectors

In [154]:
# initiate sunpath
sp = Sunpath(longitude=4.477639, latitude=51.929233)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []

#NOTE: Change day_multiple to an acceptable amount, more than 120k rays will usually run out of memory
#      on 16gb RAM
day_multiple = 75

# for every day in 365 days of a year
for d in range(365):
    # ??? filter days
    if d%day_multiple == 0:
        # for every hour in 24 hours of a day
        for h in range(24):
            # reconstruct the 
            hoy = d*24 + h
            # compute the sun object
            sun = sp.calculate_sun_from_hoy(hoy)
            # extract the sun vector
            sun_vector = sun.sun_vector.to_array()
            # ??? remove sun vectors beneath the horizon
            if sun_vector[2] < 0.0:
                # add the hoy to the list
                hoys.append(hoy)
                # add the sun vetor to the list
                sun_vectors.append(sun_vector)

# convert sun_vectors list to numpy array                
sun_vectors = np.array(sun_vectors)
# compute the rotation matrix : (angle, axis)
Rz = tm.transformations.rotation_matrix(np.radians(36.324), [0,0,1])
# Rotate the sun vectors to match the site rotation
sun_vectors = tm.transform_points(sun_vectors, Rz)
print(sun_vectors.shape)

(61, 3)


In [155]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# add the sun locations, color orange
p.add_points( - sun_vectors * 300, color='#ffa500')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(2897.556896758803, 3276.4279142148575, 3040.6264916195596),
 (-13.99542236328125, 364.87559509277344, 129.07417249747533),
 (0.0, 0.0, 1.0)]

## 4. Compute Intersection of Sun Rays with Context Mesh

### 4.1. Preparing the List of Ray Directions and Origins

In [156]:
# constructing the sun direction from the sun vectors in a numpy array
sun_dirs = -np.array(sun_vectors)
# exract the centroids of the envelope voxels
vox_cens = envelope_lattice.centroids

# next step we need to shoot in all of the sun directions from all of the voxels, todo so, we need repeat the sun direction for the number of voxels to construct the ray_dir (which is the list of all ray directions). We need to repeat the voxels for the 

# Lists for solar access
ray_dir = []
ray_src = []

# Lists for solar envelope
ray_dir2 = []
ray_src2 = []

for v_cen in vox_cens:
    for s_dir in sun_dirs:
        ray_src.append(v_cen)
        ray_dir.append(s_dir)
        ray_dir2.append(-s_dir)
ray_src2 = ray_src


# converting the list of directions and sources to numpy array
ray_dir = np.array(ray_dir)
ray_src = np.array(ray_src)
ray_dir2 = np.array(ray_dir2)
ray_src2 = np.array(ray_src2)


# Further info: for vectorized version of this code check: https://github.com/shervinazadi/spatial_computing_workshops/blob/master/notebooks/w2_solar_envelope.ipynb

print("number of voxels to shoot rays from :",vox_cens.shape)
print("number of rays per each voxel :",sun_dirs.shape)
print("number of rays to be shot :",ray_src.shape)

print(ray_dir.shape)
print(ray_src.shape)
print(ray_dir2.shape)
print(ray_src2.shape)

number of voxels to shoot rays from : (1696, 3)
number of rays per each voxel : (61, 3)
number of rays to be shot : (103456, 3)
(103456, 3)
(103456, 3)
(103456, 3)
(103456, 3)


### 4.2. Computing the Intersection

In [157]:
# Solar Access
# computing the intersections of rays with the context mesh
tri_id, ray_id = context_mesh.ray.intersects_id(ray_origins=ray_src, ray_directions=ray_dir, multiple_hits=False)
print(tri_id, ray_id)


[2601 2601 2600 ... 2600 2157 2599] [     4      5      6 ... 103413 103446 103455]


In [158]:
# Solar Envelope
# computing the intersections of rays with the context mesh
tri_id2, ray_id2 = context_mesh.ray.intersects_id(ray_origins=ray_src2, ray_directions=ray_dir2, multiple_hits=False)
print(tri_id2, ray_id2)

[ 91 113  98 ... 308 342 807] [     0      1      2 ... 103452 103454 103455]


## 5. Aggregate Simulation Result in the Sun Access Lattice

### 5.1. Compute the percentage of time that each voxel sees the sun

In [159]:
# initializing the hits list full of zeros
hits = [0]*len(ray_dir)
hits2 = [0]*len(ray_dir)
hits_final = [0]*len(ray_dir)

# setting the rays that had an intersection to 1
for i in ray_id:
    hits[i] = 1
for i in ray_id2:
    hits2[i] = 1

# Solar Envelope: check if shadow is not cast by context by comparing the acces and envelope lists
for i in range(len(hits)):
    hits_final[i] = hits[i] - hits2[i]
    if hits_final[i] == 1:
        hits_final[i] = 0
    
sun_count = len(sun_dirs)
vox_count = len(vox_cens)

# initiating the list of ratio
vox_sun_acc = []
vox_sun_block = []
test = []

# iterate over the voxels for both access and envelope
for v_id in range(vox_count):
    # counter for the intersection
    int_count = 0
    int_count_block = 0
    # iterate over the sun rays
    for s_id in range(sun_count):
        # computing the ray id from voxel id and sun id
        r_id = s_id + v_id * sun_count

        # summing the intersections
        int_count += hits[r_id]
        int_count_block -= hits_final[r_id]

    #print(int_count_block)
    # computing the percentage of the rays that DID NOT have 
    # an intersection (aka could see the sun)
    test.append(int_count)
    sun_acc = 1 - int_count / sun_count
    sun_block = int_count_block / sun_count
    # add the ratio to list
    vox_sun_acc.append(sun_acc)
    vox_sun_block.append(sun_block)

hits = np.array(hits)
vox_sun_acc = np.array(vox_sun_acc)
vox_sun_block = np.array(vox_sun_block)

# Further info: for vectorized version of this code check: https://github.com/shervinazadi/spatial_computing_workshops/blob/master/notebooks/w2_solar_envelope.ipynb
# print(test)
print(len(vox_sun_acc))
# print(vox_sun_block)

1696


### 5.2. Store sun access information in a Lattice

In [160]:
# getting the condition of all voxels: are they inside the envelop or not
env_all_vox = envelope_lattice.flatten()

# all voxels sun access
all_vox_sun_acc = []
all_vox_sun_block = []
# v_id: voxel id in the list of only interior voxels
v_id = 0

# for all the voxels, place the interiority condition of each voxel in "vox_in"
for vox_in in env_all_vox:
    # if the voxel was inside...
    if vox_in == True:
        # read its value of sun access and append it to the list of all voxel sun access
        all_vox_sun_acc.append(vox_sun_acc[v_id])
        all_vox_sun_block.append(vox_sun_block[v_id])
        # add one to the voxel id so the next time we read the next voxel
        v_id += 1
    # if the voxel was not inside... 
    else:
        # add 0.0 for its sun access
        all_vox_sun_acc.append(0.0)
        all_vox_sun_block.append(0.0)

#print(all_vox_sun_acc)
#print(all_vox_sun_block)
# convert to array
sunacc_array = np.array(all_vox_sun_acc)
sunblock_array = np.array(all_vox_sun_block)
print(sunacc_array)
# Further info: for vectorized version of this code check: https://github.com/shervinazadi/spatial_computing_workshops/blob/master/notebooks/w2_solar_envelope.ipynb

# reshape to lattice shape
sunacc_array = sunacc_array.reshape(envelope_lattice.shape)
sunblock_array = sunblock_array.reshape(envelope_lattice.shape)

# convert to lattice
sunacc_lattice = tg.to_lattice(sunacc_array, envelope_lattice)
sunblock_lattice = tg.to_lattice(sunblock_array, envelope_lattice)


print(sunacc_lattice.size)

#print(sunblock_lattice.shape)

[0. 0. 0. ... 0. 0. 0.]
14080


### 5.3. Visualize the sun access lattice

In [161]:
# Solar Access
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()


# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = sunacc_lattice.shape

# The bottom left corner of the data set
grid.origin = sunacc_lattice.minbound

# These are the cell sizes along each axis
grid.spacing = sunacc_lattice.unit


# Add the data values to the cell data
sun_acc_flat = sunacc_lattice.flatten(order="F")
print(sun_acc_flat.min())
grid.point_arrays["Sun Access"] = sun_acc_flat # Flatten the Lattice

#print(sunacc_lattice)
#print(envelope_lattice)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
p.add_volume(grid, cmap="coolwarm", clim=[0.0, 1.0], opacity=opacity, shade=False)

# plotting
p.show(use_ipyvtk=True,)

0.0


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(2883.509388359506, 3262.3804058155606, 2924.504810722787),
 (-13.99542236328125, 364.87559509277344, 27.0),
 (0.0, 0.0, 1.0)]

In [162]:
# Solar Envelope
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = sunblock_lattice.shape
# The bottom left corner of the data set
grid.origin = sunblock_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = sunblock_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Sun Block"] = sunblock_lattice.flatten(order="F")  # Flatten the Lattice
#print(sunblock_lattice.flatten(order="F"))
# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
p.add_volume(grid, cmap="coolwarm", clim=[0.0, 0.6],opacity=opacity, shade=False)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(2883.509388359506, 3262.3804058155606, 2924.504810722787),
 (-13.99542236328125, 364.87559509277344, 27.0),
 (0.0, 0.0, 1.0)]

## 6. Save Sun Access Lattice into a CSV

In [163]:
# save the sun access latice to csv
# NOTE: in "solar_y_x_x" change _x_x to current voxel size
csv_path = os.path.relpath("data\solar_access_14_4.csv")
csv_path2 = os.path.relpath("data\solar_envelope_14_4.csv")
sunacc_lattice.to_csv(csv_path)
sunblock_lattice.to_csv(csv_path2)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


## 7. Octree System of voxel size reduction

In [166]:
########################################################################################
#Initialize variables for Octree system

#NOTE: visualize treshold values in "gh/load_lattice_csv"
vox_size = 14.4   #current voxel size
req_res = 7.2     #required voxel size after this pass
fin_res = 3.6     #required voxel size after all passes
t_val_low = 0.43  #low treshold bound
t_val_high = 0.75 #high treshold bound
og_height = 16    #The shape of the current lattice size
og_length = 44
og_width = 20

#Initializing lists and variables
current_z = 0
coord_x = 0.0
coord_y = 0.0
coord_z = 0.0
coords = []
sunblock_result = []
sunblock_final = []
sunblock_val = []

sunacc_result = []
sunacc_final = []
sunacc_val = []

fin_res_coords = []
step = []
remove_coords = []

#Finding necessary multipliers  
mult = int(vox_size / req_res)
fin_mult = int(vox_size / fin_res)
steady_mult = int(14.4 / req_res)
grand_mult = int(14.4 / fin_res)
step_mult = int(req_res/fin_res)

height = og_height*mult
fin_height = og_height*fin_mult
length = og_length*mult
width = og_width*mult
########################################################################################


########################################################################################
#Convert CSV strings to integers or floats where possible
def conv(s):
    try:
        s=float(s)
        s=int(s)
    except ValueError:
        pass    
    return s
def conv1(s):
    try:
        s=float(s)
    except ValueError:
        pass    
    return s
########################################################################################


########################################################################################

#Loading classically for shape
lattice_path = os.path.relpath("data/solar_envelope_14_4.csv")
sunblock_shape = tg.lattice_from_csv(lattice_path)

#Loading solar envelope directly for further control
#NOTE: load file of current pass
with open("data/solar_envelope_14_4.csv", newline='') as f:
    reader = csv.reader(f)
    sunblock_list = list(reader)

#Converting string to int or float
for i in range(len(sunblock_list)):
    for j in range(len(sunblock_list[i])):
        step.append(conv1(sunblock_list[i][j]))
    sunblock_result.append(step)
    step = []

#Removing empty lists and first lines
sunblock_final = [x for x in sunblock_result if x != []]
for i in range(0, 5):
    sunblock_final.pop(0)


for i in range(len(sunblock_final)):
    sunblock_val.append(sunblock_final[i][3])

sunblock_val = np.array(sunblock_val)
sunblock_val = sunblock_val.reshape(sunblock_shape.shape)

#Loading solar access directly for further control
#NOTE: load file of current pass
with open("data/solar_radiance_14_4.csv", newline='') as f:
    reader = csv.reader(f)
    sunacc_list = list(reader)

#Converting string to int or float
for i in range(len(sunacc_list)):
    for j in range(len(sunacc_list[i])):
        step.append(conv1(sunacc_list[i][j]))
    sunacc_result.append(step)
    step = []

#Removing empty lists and first lines
sunacc_final = [x for x in sunacc_result if x != []]
for i in range(0, 5):
    sunacc_final.pop(0)

for i in range(len(sunacc_final)):
    sunacc_val.append(sunacc_final[i][3])

sunacc_val = np.array(sunacc_val)
sunacc_val = sunacc_val.reshape(sunblock_shape.shape)
########################################################################################


########################################################################################
#Use T_val to determine cells to recalculate, keep and remove. 
#Initialize variables
min_max_check = []
min_max_val = []
min_max_result = []
rest_coords = []
remove_coords=[]
count = 0
max_val = 0
min_val = 0

#Split voxels in to three lists, recalculate, keep and remove respectively 
for i in range(len(sunblock_val)):
    for j in range(len(sunblock_val[0])):
        for k in range(len(sunblock_val[0][0])):
            if t_val_high > sunblock_val[i][j][k] > t_val_low:
                for l  in range(0, mult):
                    coord_x = i * mult + l
                    for m in range(0, mult):
                        coord_y = j * mult + m
                        for n in range(0, mult):
                            coord_z = k * mult + n
                            coords.append([coord_x,coord_y,coord_z])
            elif t_val_high > sunblock_val[i][j][k] > 0:
                for l  in range(0, mult):
                    coord_x = i * mult + l
                    for m in range(0, mult):
                        coord_y = j * mult + m
                        for n in range(0, mult):
                            coord_z = k * mult + n
                            rest_coords.append([coord_x,coord_y,coord_z])
            elif sunblock_val[i][j][k] > t_val_high:
                for l  in range(0, mult):
                    coord_x = i * mult + l
                    for m in range(0, mult):
                        coord_y = j * mult + m
                        for n in range(0, mult):
                            coord_z = k * mult + n
                            remove_coords.append([coord_x,coord_y,coord_z])

#Find min and max z-value for each x,y in recalculate
for i in range(0, length):
    for j in range(0, width):
        for k in range(0, height):
            if [i, j, k] in coords:
                min_max_val = coords.index([i, j, k])
                min_max_check.append(coords[min_max_val][2])
        
        #create a list of [x, y, z_min, z_max]
        if len(min_max_check) > 0:
            max_val = max(min_max_check)
            min_val = min(min_max_check)
            min_max_result.append([i, j, min_val, max_val])
            min_max_check = []
            
#NOTE: When pillarforming is experienced this is due to z_min == z_max,
#      following can be enabled in later passes to prevent this.
#Force space between min and max
#for i in range(len(min_max_result)):
    #if min_max_result[i][2]+1 == min_max_result[i][3]:
        #min_max_result[i][2] = (min_max_result[i][2]-4)
#print(min_max_result)

#Creating synchronous [x,y] and [z_min, z_max] lists
min_max_only_xy = []
min_max_only_val = []
for i in range(len(min_max_result)):
    min_max_only_xy.append([min_max_result[i][0], min_max_result[i][1]])
    min_max_only_val.append([min_max_result[i][2], min_max_result[i][3]])

#Intialize next step    
full_coords = []
keep_coords = []

#Add any voxel between z_min and z_max to keep list
for i in range(len(min_max_result)):
    for j in range(min_max_result[i][2], min_max_result[i][3]+1):
        keep_coords.append([min_max_result[i][0], min_max_result[i][1], j])

#Creating a list consisting of recalculate and keep
full_coords = coords[:]
for i in rest_coords:
    full_coords.append(i)

#Initialize next step
pop_list = []
final_coords = []
temp_coords = []

#remove any z > z_min from keep list
for i in range(len(full_coords)):
    x = full_coords[i][0]
    y = full_coords[i][1]
    z = full_coords[i][2]
    if [x, y] in min_max_only_xy:
        index = min_max_only_xy.index([x, y])
        if z >= min_max_only_val[index][0]:
            pop_list.append(i)

for i in pop_list:
    full_coords[i] = 0
final_coords = [x for x in full_coords if x != 0]
temp_coords = final_coords[:]
final_coords = [x for x in temp_coords if x not in remove_coords]           
########################################################################################


########################################################################################
#Raise the resolution of voxels to keep to the final size
calc_coords = final_coords[:]
fin_res_coords = []
for i in range(len(calc_coords)):
    for j in range(0, step_mult):
        x_coord = calc_coords[i][0] * step_mult + j
        for k in range(0, step_mult):
            y_coord = calc_coords[i][1] * step_mult + k
            for l in range(0, step_mult):
                z_coord = calc_coords[i][2] * step_mult + l
                fin_res_coords.append([x_coord, y_coord, z_coord])
                
########################################################################################

[]
[[0, 6, 6], [0, 7, 6], [0, 8, 6], [0, 9, 6], [0, 10, 6], [0, 11, 6], [1, 6, 6], [1, 6, 7], [1, 7, 6], [1, 8, 6], [1, 8, 7], [1, 9, 6], [1, 10, 6], [1, 11, 6], [1, 12, 6], [2, 8, 6], [2, 8, 7], [2, 9, 6], [2, 9, 7], [2, 10, 6], [2, 11, 6], [2, 12, 6], [3, 8, 6], [3, 8, 7], [3, 9, 6], [3, 9, 7], [3, 10, 6], [3, 10, 7], [3, 11, 6], [3, 11, 7], [3, 12, 6], [3, 12, 7], [4, 0, 10], [4, 1, 10], [4, 2, 8], [4, 2, 9], [4, 3, 8], [4, 12, 6], [4, 12, 7], [5, 2, 8], [5, 2, 9], [5, 3, 8], [5, 3, 9], [5, 12, 6], [5, 12, 7], [5, 13, 6], [6, 1, 10], [6, 2, 8], [6, 2, 9], [6, 3, 8], [6, 3, 9], [6, 12, 6], [6, 12, 7], [6, 13, 6], [7, 0, 10], [7, 1, 10], [7, 2, 8], [7, 2, 9], [7, 2, 10], [7, 2, 11], [7, 3, 8], [7, 3, 9], [7, 3, 10], [7, 3, 11], [7, 12, 6], [7, 12, 7], [8, 0, 10], [8, 0, 11], [8, 1, 10], [8, 2, 10], [8, 2, 11], [8, 3, 10], [8, 3, 11], [8, 4, 8], [8, 4, 9], [8, 4, 10], [8, 4, 11], [8, 5, 8], [8, 5, 9], [8, 5, 10], [8, 6, 8], [8, 6, 9], [8, 6, 10], [8, 7, 8], [8, 7, 9], [8, 7, 10], [8, 1

In [167]:
########################################################################################
#Initialize variables
x_coord = 0.0
y_coord = 0.0
z_coord = 0.0
csv_list = []
csv_max1 = []
csv_max2 = []
temp1 = []
temp2 = []
temp3 = []
temp4 = []
list_result = []
########################################################################################


########################################################################################
#Load the 14_4 lattice and make a useable list
#Note: This list is a reference list for csv layout and does not need to be changed for each pass
with open("data/lattice_14_4.csv", newline='') as f:
    reader = csv.reader(f)
    csv_list = list(reader)

#Set values to int or float
for i in range(len(csv_list)):
    for j in range(len(csv_list[i])):
        temp1.append(conv(csv_list[i][j]))
    temp2.append(temp1)
    temp1 = []
    
#Set values to int or float
for i in range(len(csv_list)):
    for j in range(len(csv_list[i])):
        temp3.append(conv(csv_list[i][j]))
    temp4.append(temp3)
    temp3 = []

#Remove Empty Lists
csv_max1 = [x for x in temp2 if x != []]
csv_max2 = [x for x in temp4 if x != []]

#Initialize temporary variables
temp1=[]
temp2=[]
temp3=[]
temp4=[]
########################################################################################


########################################################################################
#Raising the resolution of current pass solar_envelope and solar_access
sunblock_enhanced = np.kron(sunblock_val, np.ones((fin_mult, fin_mult, fin_mult)))
for i in range(len(sunblock_enhanced)):
    for j in range(len(sunblock_enhanced[0])):
        for k in range(len(sunblock_enhanced[0][0])):
            if [i, j, k] not in fin_res_coords:
                sunblock_enhanced[i][j][k] = 2
                
sunacc_enhanced = np.kron(sunacc_val, np.ones((fin_mult, fin_mult, fin_mult)))
for i in range(len(sunacc_enhanced)):
    for j in range(len(sunacc_enhanced[0])):
        for k in range(len(sunacc_enhanced[0][0])):
            if [i, j, k] not in fin_res_coords:
                sunacc_enhanced[i][j][k] = 2

#NOTE: change filenames to current pass 
with open("data/solar_envelope_pass_1.txt", "wb") as fp:   #Pickling
    pickle.dump(sunblock_enhanced, fp)
    
with open("data/solar_access_pass_1.txt", "wb") as fp:   #Pickling
    pickle.dump(sunacc_enhanced, fp)



#print(sunblock_enhanced)
#print(sunblock_val[0][0][0])
#print(sunblock_enhanced[0][0][0], sunblock_enhanced[0][0][1], sunblock_enhanced[2][0][1],)

########################################################################################
#Saving Voxels to keep in fin_res size
#Treat top lines seperately
for i in range(0, 5):
    temp1.append(csv_max1[i])

#Change current size to size of final pass and setting both top and bottom correct
for i in range(len(csv_max1)):
    if len(csv_max1[i]) == 3 and isinstance(csv_max1[i][2], int):
        temp1[i][2] = fin_res
        temp1[i][1] = csv_max1[i][1] * grand_mult
    elif len(csv_max1[i]) == 4 and isinstance(csv_max1[i][2], int):
        for j in range(0, grand_mult):
            x_coord = csv_max1[i][0] * grand_mult + j
            for k in range(0, grand_mult):
                y_coord = csv_max1[i][1] * grand_mult + k
                for l in range(0, grand_mult):
                    z_coord = csv_max1[i][2] * grand_mult + l
                    temp2.append([x_coord, y_coord, z_coord, False])
                    
#Checking which cells to keep by setting True
for x in fin_res_coords:
    if (x+[False]) in temp2:
        i = temp2.index(x + [False])
        temp2[i][3] = True
fin_result = temp1 + temp2
print(fin_result)

temp1 = []
temp2 = []

########################################################################################


########################################################################################
#Process CSV Data for choosing voxels for further calculations
#Treat top lines seperately
for i in range(0, 5):
    temp1.append(csv_max2[i])

#Change current size to size of next pass and setting both top and bottom correct
for i in range(len(csv_max2)):
    if len(csv_max2[i]) == 3 and isinstance(csv_max2[i][2], int):
        temp1[i][2] = req_res
        temp1[i][1] = csv_max2[i][1] * steady_mult
    elif len(csv_max2[i]) == 4 and isinstance(csv_max2[i][2], int):
        for j in range(0, steady_mult):
            x_coord = csv_max2[i][0] * steady_mult + j
            for k in range(0, steady_mult):
                y_coord = csv_max2[i][1] * steady_mult + k
                for l in range(0, steady_mult):
                    z_coord = csv_max2[i][2] * steady_mult + l
                    temp2.append([x_coord, y_coord, z_coord, False])
                    
#Checking which cells to keep by setting True
for x in keep_coords:
    if (x+[False]) in temp2:
        i = temp2.index(x + [False])
        temp2[i][3] = True 
list_result = temp1 + temp2

temp1 = []
temp2 = []

########################################################################################
print("done")

[['minbound', 'shape', 'unit'], [-28, 44, 3.6], [-86, 20, 3.6], [0, 16, 3.6], ['IX', 'IY', 'IZ', 'value'], [0, 0, 0, False], [0, 0, 1, False], [0, 0, 2, False], [0, 0, 3, False], [0, 0, 4, False], [0, 0, 5, False], [0, 0, 6, False], [0, 0, 7, False], [0, 0, 8, False], [0, 0, 9, False], [0, 0, 10, False], [0, 0, 11, False], [0, 0, 12, False], [0, 0, 13, False], [0, 0, 14, False], [0, 0, 15, False], [0, 1, 0, False], [0, 1, 1, False], [0, 1, 2, False], [0, 1, 3, False], [0, 1, 4, False], [0, 1, 5, False], [0, 1, 6, False], [0, 1, 7, False], [0, 1, 8, False], [0, 1, 9, False], [0, 1, 10, False], [0, 1, 11, False], [0, 1, 12, False], [0, 1, 13, False], [0, 1, 14, False], [0, 1, 15, False], [0, 2, 0, False], [0, 2, 1, False], [0, 2, 2, False], [0, 2, 3, False], [0, 2, 4, False], [0, 2, 5, False], [0, 2, 6, False], [0, 2, 7, False], [0, 2, 8, False], [0, 2, 9, False], [0, 2, 10, False], [0, 2, 11, False], [0, 2, 12, False], [0, 2, 13, False], [0, 2, 14, False], [0, 2, 15, False], [0, 3, 0, F

done


In [168]:
########################################################################################
#Re-writing the translated and chosen data into a proper CSV
#NOTE: Change filename to current pass
with open("data/lattice_print_pass_1.csv",'w') as f:
    for row in fin_result:
        for x in range(len(row)):
            if x < (len(row)-1):
                f.write(str(row[x]) + ',')
            else:
                f.write(str(row[x]))
        f.write('\n'*2)
        
#Writing CSV with which to move to the next pass        
with open("data/lattice_continue.csv",'w') as f:
    for row in list_result:
        for x in range(len(row)):
            if x < (len(row)-1):
                f.write(str(row[x]) + ',')
            else:
                f.write(str(row[x]))
        f.write('\n'*2)
        
########################################################################################

In [73]:
#Next block not necesarry, but usefull for visualizing the results with the chosen t_vals
# loading the lattice from csv
lattice_path = os.path.relpath("data\lattice_print_pass_1.csv")
med_res_lattice = tg.lattice_from_csv(lattice_path)
#print(med_res_lattice)

KeyError: 'value'

In [68]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
med_res_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(2883.595633029055, 3262.4666504851098, 2924.5910550347085),
 (-13.99542236328125, 364.87559509277344, 26.99999964237213),
 (0.0, 0.0, 1.0)]

### Credits

In [21]:
__author__ = "Shervin Azadi and Pirouz Nourian"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Solar Envelope"